In [1]:
%load_ext autoreload
%autoreload 2

First clustering test, with kmeans and dbscan on the specific context: vary low variance, no relevant clusters are detected.
✅ Perché la varianza è bassa nel tuo caso
Stai analizzando comportamenti in contesti molto specifici (es. "1st serve su terra").

In questi contesti, le strategie sono naturalmente simili tra giocatori: tutti affrontano condizioni e dinamiche simili.

Quindi è normale che le feature (rate di slice, vincenti, lunghezza media degli scambi, ecc.) abbiano variazioni minime, da cui una PCA schiacciata e clustering difficile.

🧭 Strategia: allargare progressivamente il contesto
Ottima idea. Puoi procedere così:

1. Contesto medio-specializzato
Es. "tutti i punti al servizio su terra", unendo 1st e 2nd.

2. Contesto più ampio
Es. "tutti i punti al servizio" indipendentemente dalla superficie.

3. Contesto generale
Tutti i colpi del giocatore, aggregando tutto (serve/response, superfici).

🎯 Obiettivo finale:
Capire a che livello di granularità emergono:

differenze significative tra stili (più cluster distinti, silhouette score alto)

oppure confermare che alcuni contesti sono intrinsecamente “uniformanti” (→ bassa varianza, stile unico richiesto).

Running clustering:

In [ ]:
from run_clustering import *
import pandas as pd

# prova kmeans con vari k, stampando i silhouette scores
# decente ma meglio agglomerative
#run_kmeans_clustering(df_hard_serve)

#male
#run_dbscan_clustering(df_hard)

#gmm: risultati strani, appartenenza alta ma in optics tanto rumore... bah
#for k in range(2, 4):
#    run_umap_gmm_clustering(df_hard_serve,None,k,15,0.1,0.9,True) #todo: capire perche gmm ha max_probs altissime ma silhouette sempre basso
df_clustered=run_umap_agglomerative_clustering(df_hard_serve, None, 3, 2, 'complete', True)

#problema di optics: troppi punti di rumore, troppi giocatori rimangono esclusi
for min_pts in range(4, 11):
   for xi in np.arange(start=0.04, stop=0.12, step=0.01):
      df_clustering = run_umap_optics_clustering(df_hard, min_pts=min_pts, xi=xi,min_cluster_size=0.1)
#run_umap_optics_clustering(df_hard_serve, 'umap', None, 8, 0.08, 0.1, True)


print("Righe: ",df_hard.shape[0], "colonne: ", df_hard.shape[1])

Uploading of all the datasets:

In [ ]:
import os
import pandas as pd

dfs=[]
features_dataset_dir = '../feature_datasets_csv_reduced_contexts&features/'

# Itera su tutti i file nella cartella specificata
for dataset_name in os.listdir(features_dataset_dir):
    # Controlla se il file è un CSV
    if dataset_name.endswith('.csv'):
        # Costruisci il percorso completo del file
        file_path = os.path.join(features_dataset_dir, dataset_name)
        # Carica il dataset
        dfs.append((dataset_name,pd.read_csv(file_path, low_memory=False)))

UMAP does not preserve the convexity structure.

KMeans clustering:

In [ ]:
from run_clustering import *

dfs_clustered = []

for name, dataframe in dfs:
    print(name)
    for umap_comp in range(2, 3):
        metrics = []
        df_clustered, silhouette_score, dbi=run_kmeans_clustering(dataframe, 4, True, umap_comp)
        dfs_clustered.append((name,df_clustered))
        metrics.append((silhouette_score,dbi,umap_comp))
    print({name: max(metrics, key = lambda x: x[0])})


Agglomerative clustering:

In [ ]:
from run_clustering import *

for name, dataframe in dfs:
    print(name)
    for umap_comp in range(2, 11):
        silhouette_scores = []
        df_clustered, silhouette_score=run_umap_agglomerative_clustering(dataframe, None, 4, umap_comp, 'complete', False)
        silhouette_scores.append((silhouette_score,umap_comp))
    print({name: max(silhouette_scores, key = lambda x: x[1])})

Optics reachability plot:

In [ ]:
from run_clustering import *

for name, dataframe in dfs:
    print(name)
    for min_pts in range(6, 11):
        for xi in np.arange(start=0.06, stop=0.12, step=0.01):
            df_clustering = run_umap_optics_clustering(dataframe, min_pts=min_pts, xi=xi)

GMM cluster:

In [ ]:
from run_clustering import run_umap_gmm_clustering
for name, dataframe in dfs:
    print(name)
    for k in range(4, 5):
        run_umap_gmm_clustering(dataframe,None,k,15,0.1,0.9,True)


Agglomerate statistic by cluster visualization:

In [ ]:
from visualize_results import cluster_feature_deltas
from visualize_results import aggregate_features_by_cluster
import seaborn as sns
import matplotlib.pyplot as plt
# stampa le features aggregate per cluster
aggregated = aggregate_features_by_cluster(df_clustered)
print(aggregated)

# stampa la hitmap della differenza nelle statistiche tra i clusters e la media globale
sns.heatmap(cluster_feature_deltas(df_clustered), annot=True, cmap='coolwarm')
plt.title("Scostamenti medi delle feature per cluster")
plt.show()


Save dfs clustered in files.

In [ ]:
import os

features_dataset_clustered_dir = '../feature_datasets_csv_reduced_contexts&features_clustered/'
for name, df_clustered in dfs_clustered:
    file_path = os.path.join(features_dataset_clustered_dir, name)
    df_clustered.to_csv(file_path, index=False)

Upload dfs clustered.

In [2]:
import os
import pandas as pd

features_dataset_clustered_dir = '../feature_datasets_csv_reduced_contexts&features_clustered/'
dfs_clustered = []
for dataset_name in os.listdir(features_dataset_clustered_dir):
    if dataset_name.endswith('.csv'):
        file_path = os.path.join(features_dataset_clustered_dir, dataset_name)
        dfs_clustered.append((dataset_name,pd.read_csv(file_path)))

Calculate for each context, for each cluster, the top n representative features, i.e. the ones that have the highest difference between the cluster centroid feature and the overall feature mean.

In [10]:
from visualize_results import analyze_cluster_profiles

for name, df_clustered in dfs_clustered:
    print(name)
    analyze_cluster_profiles(df_clustered, context=name.replace(".csv", ""))

on response_ on clay.csv

--- Cluster 0 ---
 Defensive, consistent from the baseline.
  average_shot_length: - 0.0204
  net_points_rate: - 0.0455
  net_points_won_rate: - 0.1245
  winners_rate: - 0.1318
  unforced_errors_rate: - 0.0302
  slices_rate: - 0.1738
  dropshots_rate: - 0.2849
  average_response_depth: - 0.0099

Top players in the cluster 0:
  - Alexander_Zverev
  - Casper_Ruud
  - Jack_Draper
  - Jannik_Sinner
  - Taylor_Fritz

--- Cluster 1 ---
 Aggressive since the response, variety of shots, high number of net discents
  average_shot_length: - 0.1342
  net_points_rate: + 0.1052
  net_points_won_rate: - 0.0163
  winners_rate: + 0.1941
  unforced_errors_rate: + 0.0047
  slices_rate: + 0.3277
  dropshots_rate: + 0.0756
  average_response_depth: + 0.0129

Top players in the cluster 1:
  - Andrey_Rublev
  - Arthur_Fils
  - Ben_Shelton
  - Novak_Djokovic
  - Grigor_Dimitrov

--- Cluster 2 ---
 High-risk, aggressive baseliner, high usage of dropshot, not good responder to the ser

Visualize similarity matrix and heatmap. The similarity matrix calculate the distance between each cluster in a context with each other cluster in different context. For each cluster, the 3 most similar clusters in each the 3 surfaces represent the same style

In [ ]:
from visualize_results import calculate_centroid_similarity
# 3. Calcolo della Similarità tra Centroidi
clustered_data_on_serve = []
clustered_data_on_response = []
for context, df_clustered in dfs_clustered:
    if "on serve" in context:
        clustered_data_on_serve.append((context, df_clustered))
    else:
        clustered_data_on_response.append((context, df_clustered))
similarity_df_on_serve = calculate_centroid_similarity(clustered_data_on_serve)
similarity_df_on_response = calculate_centroid_similarity(clustered_data_on_response)

In [ ]:
from visualize_results import visualize_similarity_matrix

# 4. Visualizzazione della Matrice di Similarità
visualize_similarity_matrix(similarity_df_on_serve, clustered_data_on_serve)
visualize_similarity_matrix(similarity_df_on_response, clustered_data_on_response)

Calculate for each point context (serve, response) 4 triplets where 4 is the number of playing styles. Each triplet is composed by a cluster for each surface. The goal is to find the best similarity of each cluster of a surface with the others clusters of the other surfaces.

In [ ]:
#todo probabilmente non serve a niente, modificare in modo tale da stampare per ogni contest/cluster i due cluster sulle diverse superfici piu simili
from visualize_results import find_closest_cluster_triplets
on_serve_contexts = [context for context, _ in dfs_clustered if "on serve" in context]
on_response_contexts = [context for context, _ in dfs_clustered if "on response" in context]
print(find_closest_cluster_triplets(similarity_df_on_serve, on_serve_contexts))
print(find_closest_cluster_triplets(similarity_df_on_response, on_response_contexts))


Visualize top players trajectories.

In [ ]:
from visualize_results import create_player_trajectories, display_top_player_trajectories

display_top_player_trajectories(create_player_trajectories(dfs_clustered))

Visualize player trajectory by name.

In [3]:
from visualize_results import create_player_trajectories, visualize_player_trajectory
visualize_player_trajectory(create_player_trajectories(dfs_clustered), "Roger_Federer", dfs_clustered, top_features=True)

SyntaxError: ':' expected after dictionary key (visualize_results.py, line 15)

Variance visualization and hopkins statistic. A 0.75 Hopkins stat indicates clustering tendency with a 90% confidence level.

In [ ]:
from sklearn.preprocessing import StandardScaler
from run_clustering import hopkins
for name, dataframe in dfs:
    print(name)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(dataframe.values)
    dataframe.var().sort_values()
    mean, std_dev, values = hopkins(X_scaled)
    print(f"Hopkins statistic (df): {mean:.3f} ± {std_dev:.3f}\n")



Variance explained after PCA:

In [ ]:
from run_clustering import explained_variance_from_features

explained_variance_from_features(df)


The first 16 components explain more than 98% of the variance, maybe i can drop the last 4.